# project for openstreet map exploration!!

In [1]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
from collections import defaultdict

In [2]:
OSM_PATH = "DC.osm"

In [3]:
## how many varibles the attributes K has
VARIABLES=[]
VALUE={}

In [1]:
## get the variables in k
for event,elem in ET.iterparse(OSM_PATH,events=("start",)):
    if elem.tag=="node":
        for tag in elem.iter("tag"):
            x=tag.attrib['k']    
            if x not in VARIABLES:
                VARIABLES.append(x)
            
## get the possible value for each variable:
for i, attribute in enumerate(VARIABLES):
    possiblevalues=[]
    for event,elem in ET.iterparse(OSM_PATH,events=("start",)):
        if elem.tag=="node":
            for tag in elem.iter("tag"):
                if tag.attrib['k']==attribute:
                    x=tag.attrib['v']  
                    if x not in possiblevalues:
                        possiblevalues.append(x)
    VALUE[attribute]=possiblevalues
            
            
                

'for i, attribute in enumerate(VARIABLES):\n    possiblevalues=[]\n    for event,elem in ET.iterparse(OSM_PATH,events=("start",)):\n        if elem.tag=="node":\n            for tag in elem.iter("tag"):\n                if tag.attrib[\'k\']==attribute:\n                    x=tag.attrib[\'v\']  \n                    if x not in possiblevalues:\n                        possiblevalues.append(x)\n    VALUE[attribute]=possiblevalues\n            '

### all possible values for possible key

In [2]:
VALUE

### audit the state name

In [235]:
expected_state=['VA','DC','MD']

In [236]:
def audit_state_type(state_name):
    if state_name not in expected_state:
         state_names.append(state_name)

In [3]:
VALUE['addr:state']
['Virginia', 'MD', 'DC', 'VA', 'va', 'dc']
for i ,name in enumerate (VALUE['addr:state']):
    audit_state_type(name)
state_names

### audit the street name 

In [238]:
expected_street_end=["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons",'Northwest','Southeast','Northeast','Southwest','South','North','East','West',
            'Circle','Way','Highway','Pike','Plaza']
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

In [239]:
def audit_street_type(street_name):
    m=street_type_re.search(street_name)
    if m:
        street_type=m.group()
        if street_type not in expected_street_end:
            street_types[street_type].add(street_name)

In [240]:
for i ,name in enumerate (VALUE['addr:street']):
    audit_street_type(name)

In [241]:
street_types

defaultdict(set,
            {'1': {'1'},
             '100': {'L St NW #100'},
             '2': {'South Glebe Road, Bldg. 2'},
             'Hill': {'Capitol Hill'},
             'Hwy': {'Jefferson Davis Hwy'},
             'Ln': {'Slaters Ln'},
             'Mews': {'North Glebe Mews'},
             'NE': {'H St NE', 'Minnesota Avenue NE'},
             'NW': {'Connecticut Ave NW',
              'M Street NW',
              'New Jersey Avenue NW',
              'Pennsylvania Avenue NW',
              'Wisconsin Avenue, NW'},
             'SE': {'Pennsylvania Avenue SE'},
             'SW': {'Half Street SW',
              'Maryland Avenue SW',
              'South CapitoL Street SW'},
             'St': {'N. Rhodes St', 'North Stafford St'}})

### cusine type

In [242]:
#VALUE['cuisine']

### opening_hours

In [243]:
#VALUE['opening_hours']

### prepare data ready for sql import

##### schema

In [5]:
schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'node_openhours': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'hour': {'required': True, 'type': 'string'},
                'section':{'required': True, 'type': 'string'}
            }
        }
    },
    
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
     'way_openhours': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'hour': {'required': True, 'type': 'string'},
                'section':{'required': True, 'type': 'string'}
            }
        }
    },
    
}




#### dataframe for tables

In [6]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"
NODE_TAGS_OPENING="node_opening_hours.csv"
WAY_TAGS_OPENING='way_opening_hours.csv'

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']
NODE_TAGS_OPENING_FIELDS=['id','hour','section']
WAY_TAGS_OPENING_FIELDS=['id','hour','section']

#### helper functions

In [7]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


#### update functions for states and streets

In [8]:
#### update the state name
mapping_states={'Virginia':'VA', 'va':'VA', 'dc':'DC'}
correct_ones=['VA','DC','MD']
def update_states_name(name):
    if name not in correct_ones:
        name=mapping_states[name]
            ##replacedString = re.sub(pattern, replacement_pattern, input_str, count, flags=0)
    return name
##### update the street name
expected_street_end=["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons",'Northwest','Southeast','Northeast','Southwest','South','North','East','West',
            'Circle','Way','Highway','Pike','Plaza','Hill','Mews']


mapping_street={
             'Hwy': 'Highway',
             'Ln': 'Lane',
             'NE': 'Northeast',
             'NW': 'Northwest',
             'SE': 'Southeast',
             'SW': 'Southwest',
             'St': 'Street',
             'Ave': 'Avenue'}
def update_street_name(name):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected_street_end:
            if street_type in mapping_street.keys():
                name = re.sub(street_type_re, mapping_street[street_type], name)
            ##replacedString = re.sub(pattern, replacement_pattern, input_str, count, flags=0)
    return name

#### shape element function

In [9]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    opening_hours=[]

    # YOUR CODE HERE
    if element.tag=='node' :
        for node in NODE_FIELDS:
            node_attribs[node] = element.attrib[node]   
        for child in element:
            if child.tag=="tag":
                
                tag = {}
                #print node_attribs['id']
                tag['id'] = node_attribs['id']
                tag['value'] = child.attrib['v']
    
                problem = PROBLEMCHARS.search(child.attrib['k'])
                match = LOWER_COLON.search(child.attrib['k'])
                if child.attrib['k']=='opening_hours':
                    opening={}
                    value_total=child.attrib['v']
                    for i,value_hour in enumerate(value_total.split(";")):
                        opening['id']=node_attribs['id']
                        opening['section']=str(i)
                        opening['hour']=value_hour
                    opening_hours.append(opening)
                elif problem:
                    p=problem.group()
                    #print child.attrib['k']
                    tag_key = p
                    tag['key'] = tag_key
                    tag['type'] = 'problematic'
                    if tag:
                        tags.append(tag)
                        continue
                elif match:
                    m = match.group()
                    #print(m)
                    tag_type = m.split(':')[1]
                    tag_key = m.split(':')[0]
                    tag['key'] = tag_key
                    tag['type'] = tag_type
                    if tag_key=='state':
                        tag['value'] = update_states_name(child.attrib['v'])
                    elif tag_key=='street':
                        tag['value'] = update_states_name(child.attrib['v'])
                    tags.append(tag)
                else:
                    tag['key'] = child.attrib['k']
                    tag['type'] = 'other_regular'
                    if tag:
                        tags.append(tag)
                     
    # if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags,'node_openhours':opening_hours}
    elif element.tag=='way' :
        i=0
        for way in WAY_FIELDS:
            way_attribs[way] = element.attrib[way]   
        for child in element:

            if child.tag=="tag":
                
                tag = {}
                #print way_attribs['id']
                tag['id'] = way_attribs['id']
                #tag['id'] = node_attribs['id']
                tag['value'] = child.attrib['v']
    
                problem = PROBLEMCHARS.search(child.attrib['k'])
                match = LOWER_COLON.search(child.attrib['k'])
                if child.attrib['k']=='opening_hours':
                    opening={}
                    value_total=child.attrib['v']
                    for i,value_hour in enumerate(value_total.split(";")):
                        opening['id']=way_attribs['id']
                        opening['section']=str(i)
                        opening['hour']=value_hour
                    opening_hours.append(opening)
                elif problem:
                    p=problem.group()
                    #print child.attrib['k']
                    tag_key = p
                    tag['key'] = tag_key
                    tag['type'] = 'problematic'
                    if tag:
                        tags.append(tag)
                        continue
                elif match:
                    m = match.group()
                    #print(m)
                    tag_type = m.split(':')[1]
                    tag_key = m.split(':')[0]
                    tag['key'] = tag_key
                    tag['type'] = tag_type
                    if tag_key=='state':
                        tag['value'] = update_states_name(child.attrib['v'])
                    elif tag_key=='street':
                         tag['value'] = update_states_name(child.attrib['v'])
                    
                    tags.append(tag)
                else:
                    tag['key'] = child.attrib['k']
                    tag['type'] = 'other_regular'
                    if tag:
                        tags.append(tag)
                     
                      
            elif child.tag=='nd':
                no={}
                no['id']=way_attribs['id']
                no['node_id']=child.attrib['ref']
                no['position']=i
                i+=1
                way_nodes.append(no)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags,'way_openhours':opening_hours}


#### main function

In [10]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_OPENING,'w')as way_opening_file, \
         codecs.open(NODE_TAGS_OPENING,'w')as node_opening_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)
        node_tag_hour_writer=UnicodeDictWriter(node_opening_file, NODE_TAGS_OPENING_FIELDS)
        way_tag_hour_writer=UnicodeDictWriter(way_opening_file, WAY_TAGS_OPENING_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
        node_tag_hour_writer.writeheader()
        way_tag_hour_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            #print el
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                    node_tag_hour_writer.writerows(el['node_openhours'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    way_tag_hour_writer.writerows(el['way_openhours'])
                    #print len(el['way_nodes']),len(el['way_tags'])

In [11]:
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)
